<a href="https://colab.research.google.com/github/gzholtkevych/LambdaLearning/blob/main/PureCalculus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pure $\lambda$-Calculus**

[The deatailed script](https://www.mathcha.io/editor/Pvvz5UZ1t7ktL6sZJYp19sZnX9vVserJMEKhJvvMx7)

## **Variables**

The code below models variables.

Using the `natgen()` generator in this code ensures that a fresh variable is returned in response to each constructor call.

In [1]:
def natgen():
    n = 0
    while True:
        yield n
        n += 1


class Var:
    __nats = natgen()

    def __init__(self):
        self._idx = next(Var.__nats)

    def __hash__(self):
        return self._idx.__hash__()

    def __str__(self):
        return "v[" + str(self._idx) + "]"

    def __eq__(self, other):
        return self._idx == other._idx

## **Terms**


In [2]:
class Term:

    @property
    def isAtom(self):
        """checks whether the term is an atom"""
        return isinstance(self, Atom)

    @property
    def isApplication(self):
        """checks whether the term is an application"""
        return isinstance(self, Application)

    @property
    def isAbstraction(self):
        """checks whether the term is an abstraction"""
        return isinstance(self, Abstraction)

    def __str__(self):
        if self.isAtom:
            return str(self._var)
        if self.isApplication:
            return "(" + str(self._sub) + " " + str(self._obj) + ")"
        # self is Abbstraction
        return "(fun " + str(self._head) + " => " + str(self._body) + ")"

    def __eq__(self, other):
        if self.isAtom and otherisAtom:
            return self._var == other._var
        if isinstance(self, Application) and isinstance(other, Application):
            return self._sub == other._sub and self._obj == other._obj
        if isinstance(self, Abstraction) and isinstance(other, Abstraction):
            return self._head == other._head and self._body == other._body

    @property
    def isBetaRedex(self):
        """checks whether the term is a beta-redex"""
        return self.isApplication and self._sub.isAbstraction

    @property
    def redexes(self):
        """determiness all beta-redexes in the term"""
        if self.isAtom:
            return []
        if self.isAbstraction:
            return self._body.redexes
        # self is Application
        temp = [self ]if self.isBetaRedex else []
        temp += (self._sub.redexes + self._obj.redexes)
        return temp


    @property
    def _vars(self):
        """
        returns
        -------
            the dictionary stuctured as follows
                dict[Var, dict[['free' | 'bound'], int]]
            Here, keys of the external dictionary are the variables that
            are occurred in 'self', and values of the internal dictionaries
            relate respectively to the numbers of free and bound occurrences
            of the variables.
        """
        if self.isAtom:
            return {self._var: {'free': 1, 'bound': 0}}
        if self.isApplication:
            vars, auxvars = dict(self._sub._vars), self._obj._vars
            for var in auxvars:
                try:
                    for key in {'free', 'bound'}:
                       vars[var][key] += self._obj._vars[var][key]
                except KeyError:
                    vars[var] = dict(self._obj._vars[var])
            return vars
        # self is Abstraction
        vars = dict(self._body._vars)
        try:
            vars[self._head]['bound'] += vars[self._head]['free']
            vars[self._head]['free'] = 0
        except KeyError:
            pass
        return vars


class Atom(Term):
    def __init__(self, x: Var):
        if isinstance(x, Var):
            self._var = x
        else:
            raise TypeError("a variable is waiting")


class Application(Term):
    def __init__(self, X : Term, Y : Term):
        if isinstance(X, Term) and isinstance(Y, Term):
            self._sub = X
            self._obj = Y
        else:
            raise TypeError("a term is waiting")


class Abstraction(Term):
    def __init__(self, x: Var, X: Term):
        if isinstance(x, Var):
            if isinstance(X, Term):
                self._head = x
                self._body = X
            else:
                raise TypeError("a term is waiting")
        else:
            raise TypeError("a variable is waiting")


In [3]:
x, y, z = Var(), Var(), Var()
X, Z = Atom(x), Atom(z)
XXX = Application(Application(X, X), X)
XZ = Application(X, Z)
T = Application(Abstraction(x, XXX),
                Abstraction(x, Application(Abstraction(y, Z),
                                           XZ
                                          ))
               )

In [4]:
print(T)
for var, item in T._vars.items():
    print("\t{}".format(var), end=": ")
    print(item)

((fun v[0] => ((v[0] v[0]) v[0])) (fun v[0] => ((fun v[1] => v[2]) (v[0] v[2]))))
	v[0]: {'free': 0, 'bound': 4}
	v[2]: {'free': 2, 'bound': 0}
